<a href="https://colab.research.google.com/github/jangInw007/dataviz2025/blob/main/midterm2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#นางสาวจิตรลดา นิมินรัมย์ 663020278-5

ให้สืบหาฆาตรกร (10 คะแนน) และ ผู้บงการ (5 คะแนน) ในคดี ฆาตรกรรม (murder) ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City โดยเริ่มจากการตรวจสอบ ข้อมูลคดีในไฟล์ crime_scene_report.csv

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# โหลดไฟล์ข้อมูล
crime = pd.read_csv("/content/drive/MyDrive/Python&DataViz2025/midterm/crime_scene_report.csv")
person = pd.read_csv("/content/drive/MyDrive/Python&DataViz2025/midterm/person.csv")
interview = pd.read_csv("/content/drive/MyDrive/Python&DataViz2025/midterm/interview.csv")
drivers = pd.read_csv("/content/drive/MyDrive/Python&DataViz2025/midterm/drivers_license.csv")
gym_member = pd.read_csv("/content/drive/MyDrive/Python&DataViz2025/midterm/get_fit_now_member.csv")
gym_checkin = pd.read_csv("/content/drive/MyDrive/Python&DataViz2025/midterm/get_fit_now_check_in.csv")
fb_event = pd.read_csv("/content/drive/MyDrive/Python&DataViz2025/midterm/facebook_event_checkin.csv")
income = pd.read_csv("/content/drive/MyDrive/Python&DataViz2025/midterm/income.csv")

In [4]:
# ดูคดี murder วันที่ 15 ม.ค. 2018 ที่ Pandas City
murder_case = crime[
    (crime["type"] == "murder") &
    (crime["date"] == 20180115) &
    (crime["city"] == "Pandas City")
]
murder_case

,date,type,description,city
1227,20180115,murder,Security footage shows that there were 2 witne...,Pandas City


ภาพจากกล้องวงจรปิดแสดงให้เห็นว่ามีพยาน 2 คน พยานคนแรกอาศัยอยู่ที่บ้านหลังสุดท้ายบนถนน "Northwestern Dr" ส่วนพยานคนที่สองชื่อแอนนาเบล อาศัยอยู่ที่ไหนสักแห่งบนถนน "Franklin Ave"

# ฆาตรกร (10 คะแนน) คือ 	Jeremy Bowers

In [5]:
# หาพยาน witnesses
#   - last house on Northwestern Dr
#   - Annabel on Franklin Ave
witness1 = person[(person["address_street_name"] == "Northwestern Dr")]
witness1 = witness1.loc[witness1["address_number"].idxmax()]
print(" Witness 1:", witness1["name"])

witness2 = person[
    (person["address_street_name"] == "Franklin Ave") &
    (person["name"].str.contains("Annabel"))
]
print(" Witness 2:", witness2["name"].values[0])

 Witness 1: Morty Schapiro
 Witness 2: Annabel Miller


ค่าที่มากที่สุดในคอลัมน์ 'address_number' เนื่องจากบ้านบนถนนมักจะมีหมายเลขเรียงกัน หมายเลขที่อยู่สูงสุดจึงสอดคล้องกับบ้านหลังสุดท้ายบนถนน

In [6]:
# ดู interview transcripts ของพยานทั้งสอง
witness_ids = [witness1["id"], witness2["id"]]

In [7]:
witness_notes1 = interview[interview["person_id"] == witness_ids[0]]
witness_notes1

,person_id,transcript
4988,14887,I heard a gunshot and then saw a man run out. ...


ฉันได้ยินเสียงปืน แล้วก็เห็นผู้ชายคนหนึ่งวิ่งออกมา เขาถือกระเป๋า "Get Fit Now Gym" หมายเลขสมาชิกบนกระเป๋าขึ้นต้นด้วย "48Z" มีแต่สมาชิกระดับ Gold เท่านั้นที่จะมีกระเป๋าแบบนั้น ชายคนนั้นขึ้นรถที่มีป้ายทะเบียน "H42W"

In [8]:
# ดู interview transcript ของพยานคนที่สอง (Annabel Miller)
# Access the scalar ID from witness2["id"]
witness_notes2 = interview[interview["person_id"] == witness2["id"].iloc[0]]

print("\n📝 Annabel Miller's Interview:")
display(witness_notes2)


📝 Annabel Miller's Interview:


,person_id,transcript
4989,16371,"I saw the murder happen, and I recognized the ..."


In [9]:
# จากเบาะแส: สมาชิก gym ที่ขึ้นต้นด้วย "48Z" และเป็น gold member
suspects = gym_member[
    (gym_member["id"].str.startswith("48Z"))&
    (gym_member["membership_status"] == "gold")
]
suspects

,id,person_id,name,membership_start_date,membership_status
181,48Z7A,28819,Joe Germuska,20160305,gold
182,48Z55,67318,Jeremy Bowers,20160101,gold


In [10]:
suspect_checkin = gym_checkin[
    (gym_checkin["check_in_date"] == 20180109) &
    (gym_checkin["membership_id"].isin(suspects["id"]))
]
suspect_checkin

,membership_id,check_in_date,check_in_time,check_out_time
2700,48Z7A,20180109,1600,1730
2701,48Z55,20180109,1530,1700


In [11]:
# หาคนขับรถที่มีป้ายทะเบียน "H42W"
driver_match = drivers[drivers["plate_number"].str.contains("H42W")]

print("\n🚗 Driver with license plate 'H42W':")
display(driver_match)


🚗 Driver with license plate 'H42W':


,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
915,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
3529,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
6240,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [12]:
# สร้าง mapping จาก license_id ไป name
license_to_name = person.set_index('license_id')['name']

In [13]:
# ใช้ .map() เพื่อเพิ่มคอลัมน์ชื่อใน driver_match
driver_match['name'] = driver_match['id'].map(license_to_name)
driver_match

/tmp/ipython-input-3208464250.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_match['name'] = driver_match['id'].map(license_to_name)


,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,name
915,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius,Maxine Whitely
3529,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,Jeremy Bowers
6240,664760,21,71,black,black,male,4H42WR,Nissan,Altima,Tushar Chandra


# ผู้บงการ (5 คะแนน) คือ Miranda Priestly

In [14]:
# ตรวจสอบบันทึกการสัมภาษณ์ของ Jeremy Bowers
jeremy_bowers_id = suspects[suspects["name"] == "Jeremy Bowers"]["person_id"].values[0]
jeremy_bowers_interview = interview[interview["person_id"] == jeremy_bowers_id]

print("\n📝 Jeremy Bowers' Interview:")
display(jeremy_bowers_interview)


📝 Jeremy Bowers' Interview:


,person_id,transcript
4990,67318,I was hired by a woman with a lot of money. I ...


ฉันถูกจ้างโดยผู้หญิงคนหนึ่งที่มีเงินมากมาย ฉันไม่รู้จักชื่อเธอ แต่ฉันรู้ว่าเธอสูงประมาณ 5 ฟุต 5 นิ้ว (65 นิ้ว) หรือ 5 ฟุต 7 นิ้ว (67 นิ้ว) เธอมีผมสีแดงและขับรถ Tesla Model S ฉันรู้ว่าเธอไปคอนเสิร์ต SQL Symphony 3 ครั้งในเดือนธันวาคม 2017

In [15]:
# ค้นหาผู้ต้องสงสัยในฐานะผู้อยู่เบื้องหลังจากเบาะแสใบขับขี่
mastermind_candidates = drivers[
    (drivers['gender'] == 'female') &
    (drivers['hair_color'] == 'red') &
    (drivers['car_make'] == 'Tesla') &
    (drivers['car_model'] == 'Model S')
]
mastermind_candidates

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
1105,202298,68,66,green,red,female,500123,Tesla,Model S
2054,291182,65,66,blue,red,female,08CM64,Tesla,Model S
9078,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [16]:
# ค้นหาผู้บงการโดยรวมเงื่อนไขทั้งหมด (ใช้ .map)
# a. กรองตามเบาะแสใบขับขี่: เพศหญิง, ผมสีแดง, รถ Tesla Model S
mastermind_drivers_map = drivers[
    (drivers['gender'] == 'female') &
    (drivers['hair_color'] == 'red') &
    (drivers['car_make'] == 'Tesla') &
    (drivers['car_model'] == 'Model S')
].copy() # ใช้ .copy() เพื่อหลีกเลี่ยง SettingWithCopyWarning

# สร้าง mapping จาก license_id ไป name
person_id_to_name_map = person.set_index('license_id')['name']

# ใช้ .map() เพื่อเพิ่มคอลัมน์ชื่อ
mastermind_drivers_map['name'] = mastermind_drivers_map['id'].map(person_id_to_name_map)

In [24]:
# สร้าง mapping จาก person_id ไป count จากผู้เข้าชมคอนเสิร์ต 3+ ครั้ง
# กรองข้อมูลผู้เข้าร่วมคอนเสิร์ต: เลือกข้อมูลการเช็คอินกิจกรรม (fb_event) เฉพาะกิจกรรมที่มีชื่อ "SQL Symphony Concert" และเกิดขึ้นในเดือนธันวาคม ปี 2017
sql_concert_attendees_dec_2017 = fb_event[
    (fb_event["event_name"] == "SQL Symphony Concert") &
    (fb_event["date"].astype(str).str.startswith("201712"))
]
sql_concert_attendees_dec_2017

,person_id,event_id,event_name,date
4087,62596,1143,SQL Symphony Concert,20171225
4392,19260,1143,SQL Symphony Concert,20171214
6552,58898,1143,SQL Symphony Concert,20171220
6620,69699,1143,SQL Symphony Concert,20171214
6717,19292,1143,SQL Symphony Concert,20171213
7266,43366,1143,SQL Symphony Concert,20171207
8395,92343,1143,SQL Symphony Concert,20171212
10909,28582,1143,SQL Symphony Concert,20171220
10910,28582,1143,SQL Symphony Concert,20171215
13719,81526,1143,SQL Symphony Concert,20171202


In [26]:
# นับจำนวนครั้งที่เข้าร่วม
mastermind_concert_attendees = sql_concert_attendees_dec_2017.groupby("person_id").size().reset_index(name="count")

# กรองผู้ต้องสงสัย: เลือกเฉพาะคนที่เข้าร่วมคอนเสิร์ต 3 ครั้งขึ้นไป ซึ่งตรงกับเบาะแสที่ได้มา
mastermind_concert_attendees = mastermind_concert_attendees[mastermind_concert_attendees["count"] >= 3]

In [27]:
if 'mastermind_concert_attendees' in globals():
    person_id_to_count_map = mastermind_concert_attendees.set_index('person_id')['count']

    # Use .map() เพิ่มจำนวนคอนเสิร์ตในข้อมูลผู้ขับขี่
    mastermind_drivers_map['concert_count'] = mastermind_drivers_map['id'].map(person[['license_id', 'id']].set_index('license_id')['id']).map(person_id_to_count_map)

    # กรองผู้บงการคนสุดท้าย: เลือกเฉพาะผู้ขับขี่ที่มีข้อมูลจำนวนครั้งที่เข้าร่วมคอนเสิร์ต (ไม่ใช่ค่าว่าง NaN) ซึ่งหมายความว่าพวกเขาตรงกับเงื่อนไขการไปคอนเสิร์ต 3 ครั้งขึ้นไป
    final_mastermind_candidate_map = mastermind_drivers_map.dropna(subset=['concert_count'])

    print("\n👑 Final Mastermind Candidate (using .map):")
    display(final_mastermind_candidate_map[['name', 'age', 'height', 'plate_number', 'concert_count']])
else:
    print("Error: 'mastermind_concert_attendees' dataframe not found. Please run the previous steps to identify concert attendees.")


👑 Final Mastermind Candidate (using .map):


,name,age,height,plate_number,concert_count
1105,Miranda Priestly,68,66,500123,3.0


#ในคดี ฆาตรกรรม (murder) ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City

ฆาตรกร (10 คะแนน) คือ Jeremy Bowers

ผู้บงการ (5 คะแนน) คือ Miranda Priestly